In [1]:
!pip install -q DataLoader
!pip install -q transformers
!pip install -q pytorch-crf
!pip install -q datasets
!pip install -q RobertaModel
!pip install -q seqeval
!pip install -q datasets
!pip install -q datasets transformers[sentencepiece]

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement RobertaModel (from versions: none)
ERROR: No matching distribution found for RobertaModel
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
import json
from transformers import AutoModel
import torch
from torchcrf import CRF
import torch.nn as nn
from seqeval.metrics import classification_report
from datasets import DatasetDict, Dataset
import numpy as np
from datasets import DatasetDict, Dataset
from transformers import DataCollatorForTokenClassification
from torch.utils.data import DataLoader
from transformers import DataCollatorForTokenClassification


In [3]:
from transformers import RobertaModel, RobertaConfig, RobertaTokenizer
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer
import datasets

model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"  # Mejor modelo
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)
tokenizer.is_fast
config = RobertaConfig.from_pretrained(model_checkpoint)
model = RobertaModel.from_pretrained(model_checkpoint, config=config)
num_labels = 7


# Definir la arquitectura del modelo CRF
class NERModel(nn.Module):
    def __init__(self, model, num_labels):
        super(NERModel, self).__init__()
        self.model = model
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.crf = CRF(num_labels)  # Agregar capa CRF

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        return logits

    def loss(self, logits, labels, attention_mask):
        mask = attention_mask.bool()  # Convertir la máscara a tipo booleano
        loss = -self.crf(logits, labels, mask=mask, reduction='mean')
        return loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ner_model = NERModel(model,num_labels).to(device)
config = tokenizer.model_max_length





Some weights of RobertaModel were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-biomedical-clinical-es and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from google.colab import drive
from transformers import RobertaModel, RobertaConfig, RobertaTokenizer
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# Montar Google Drive
drive.mount('/content/drive')
model_checkpoint = "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"
config = RobertaConfig.from_pretrained(model_checkpoint)
model = RobertaModel.from_pretrained(model_checkpoint, config=config)
num_labels = 7
ner_model = NERModel(model, num_labels)
ner_model.load_state_dict(torch.load("/content/drive/MyDrive/VIU/MODELO/ner_model.pth"))

label_names = ["O", "B-NegREF", "I-NegREF", "B-NEG", "I-NEG", "B-NSCO", "I-NSCO"]
def convert_labels_to_text(labels):
    return [label_names[label] for label in labels]

Mounted at /content/drive


Some weights of RobertaModel were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-biomedical-clinical-es and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [6]:
label_names = ["O", "B-NegREF", "I-NegREF", "B-NEG", "I-NEG", "B-NSCO", "I-NSCO"]
def convert_labels_to_text(labels):
    return [label_names[label] for label in labels]

In [7]:
while True:
    user_input = input("Ingresa una frase para etiquetar: ")

    # Tokenizar y procesar la entrada del usuario
    inputs = tokenizer(user_input, return_tensors="pt", truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Pasar la entrada a través del modelo
    with torch.no_grad():
        logits = ner_model(**inputs)

    # Obtener las etiquetas predichas
    predicted_labels = torch.argmax(logits, dim=2).squeeze().cpu().numpy()

    # Convertir las etiquetas a texto
    predicted_labels_text = convert_labels_to_text(predicted_labels)

    # Dividir las palabras tokenizadas y las etiquetas predichas
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # Imprimir las palabras tokenizadas y las etiquetas predichas
    for token, label in zip(tokens, predicted_labels_text):
        if token not in ['<s>', '</s>']:
            if label == 'B-NegREF':
                label = 'O'
            if label == 'I-NegREF':
                label = 'O'
            print(f"{token}: {label}")

Ingresa una frase para etiquetar: No tiene cancer pero yo si
ĠNo: B-NEG
Ġtiene: B-NSCO
Ġcancer: I-NSCO
Ġpero: O
Ġyo: O
Ġsi: O
Ingresa una frase para etiquetar: el jugador de futbol juega sin medias pero con zapatos
Ġel: O
Ġjugador: O
Ġde: O
Ġfutbol: O
Ġjuega: O
Ġsin: B-NEG
Ġmedias: B-NSCO
Ġpero: O
Ġcon: O
Ġzapatos: O


KeyboardInterrupt: ignored